<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize-extract-genQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TODO : OPTIMIZEEEEEE
# TODO : RUN MULTI TEXT

In [ ]:
# CREATE BATCH OF TEXTS

In [ ]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers==4.15.0    # only this ver works with FastT5
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git
# FAST T5
!pip3 install fastt5

In [48]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pke
import string
import re
import numpy as np

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [50]:
from fastT5 import get_onnx_model, get_onnx_runtime_sessions, OnnxT5 
from transformers import AutoTokenizer
from pathlib import Path
import os

# summarize
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-summarize'
model_name = "t5-large"
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
sum_model = OnnxT5(model_path, model_sessions)
sum_tokenizer = AutoTokenizer.from_pretrained(model_path)

# que model
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-question'
model_name = 't5_squad_v1'
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
q_model = OnnxT5(model_path, model_sessions)
q_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [150]:
def preprocess_bulk_text(text):
  text = temp_pre(text)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = sum_tokenizer.encode_plus("summarize: " + text, return_tensors='pt', max_length=512, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [231]:
def temp_pre(text):
  text = text.strip()
  text = re.sub('[\u2010-\u2013]', '-', text)
  text = re.sub('[^a-zA-Z0-9\.,-?]', ' ', text)
  text = re.sub(' {2,}', ' ', text)
  return text

In [247]:
full_text = """
 Regardless of the kind of service, cloud computing services provide users with a series of functions including:

    Email
    Storage, backup, and data retrieval
    Creating and testing apps
    Analyzing data
    Audio and video streaming
    Delivering software on demand

Cloud computing is still a fairly new service but is being used by a number of different organizations from big corporations to small businesses, nonprofits to government agencies, and even individual consumers. 
"""

# 1)

# Centralized DBMS architecture, It means files are stored in a single location and it is maintained from only that location. It is usually used by universities, companies and banks.
# Advantages of Centralized DBMS architecture, 
# The data integrity is increased. So, the data is accurate and consistent.
# Since all data in one place, data security is increased.
# It is cheaper as it requires less power and maintenance.
# Disadvantages of Centralized DBMS architecture,
# Lot of data access traffic which may bottleneck in some situations.
# If multiple users try to access it simultaneously this may reduce the efficiency of the system.
# There is no database recovery measure when system failure occurs, so all data will be lost.
# Since all data is centralized, it takes more time to access it and search. And if the network is slow, then it takes even more time.



In [175]:
full_text

'\n•\tGraph DBMS – Uses graph structure for semantic queries. Data is stored in the form of nodes, edges and properties. \nImportance, \n\uf0a7\tAdds more detail to data.\n\n•\tER DBMS – In simple RDBMS implementation a relationship between entities is implemented by storing the primary key of one entity as a foreign key in table of another entity.\nImportance,\n\uf0a7\tCreates relationship between entities between tables.\n\n•\tDocument DBMS – NoSQL databases storing database in document form. Each document represents data, its relationship between other data elements and attributes of data.\nImportance,\n\uf0a7\tAmazing document storage capability and as its NoSQL, it provides fasters mechanism to store and search documents.\n\n•\tNoSQL DBMS – Database that don’t use SQL. \nImportance,\n\uf0a7\tIt doesn’t have predefined schemas so its best for rapidly changing development environments.\n\uf0a7\tAllows developers to make changes on the fly without affecting applications.\n\n\n'

In [248]:
temp_pre(full_text)

'Regardless of the kind of service, cloud computing services provide users with a series of functions including: Email Storage, backup, and data retrieval Creating and testing apps Analyzing data Audio and video streaming Delivering software on demand Cloud computing is still a fairly new service but is being used by a number of different organizations from big corporations to small businesses, nonprofits to government agencies, and even individual consumers. Cloud can be cloud / on device.'

In [52]:
def postprocess_summary(text):
  output = ""

  for x in sent_tokenize(text):
    x = x.capitalize()
    output += " " + x

  return output

In [53]:
def summarize(text):
  input_tokens_ids, attention_mask = preprocess_bulk_text(text)
  summary_encoded = sum_model.generate(input_ids=input_tokens_ids, 
                                   attention_mask=attention_mask,
                                   num_beams=3,
                                   num_return_sequences=1,
                                   no_repeat_ngram_size=2, 
                                   max_length=300, 
                                   early_stopping=True)
  # decode summarized token
  output = sum_tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  return postprocess_summary(output)

In [54]:
def extract_keywords(text, kw_pop):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

  kw = ex.get_n_best(n=kw_pop)

  kw_ls = []

  for i in kw:
    kw_ls.append(i[0])

  return kw_ls

In [237]:
def filter_keywords(original, summarized, kw_pop=10):
  orig_ls = extract_keywords(original, kw_pop)
  sum_ls = extract_keywords(summarized, kw_pop)

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [243]:
# full_text = """
# Elon Musk has shown again he can influence the digital currency market with just his tweets. After saying that his electric vehicle-making company
# Tesla will not accept payments in Bitcoin because of environmental concerns, he tweeted that he was working with developers of Dogecoin to improve
# system transaction efficiency. Following the two distinct statements from him, the world's largest cryptocurrency hit a two-month low, while Dogecoin
# rallied by about 20 percent. The SpaceX CEO has in recent months often tweeted in support of Dogecoin, but rarely for Bitcoin.  In a recent tweet,
# Musk put out a statement from Tesla that it was “concerned” about the rapidly increasing use of fossil fuels for Bitcoin (price in India) mining and
# transaction, and hence was suspending vehicle purchases using the cryptocurrency.  A day later he again tweeted saying, “To be clear, I strongly
# believe in crypto, but it can't drive a massive increase in fossil fuel use, especially coal”.  It triggered a downward spiral for Bitcoin value but
# the cryptocurrency has stabilised since.   A number of Twitter users welcomed Musk's statement. One of them said it's time people started realising
# that Dogecoin “is here to stay” and another referred to Musk's previous assertion that crypto could become the world's future currency.
# """


summary = summarize(full_text)
print(summary)

 Cloud computing services provide users with a series of functions including: email storage, backup, and data retrieval creating and testing apps analyzing data audio and video streaming delivering software on demand.


In [244]:
filtered_kw = filter_keywords(full_text, summary)

In [245]:
filtered_kw

['series', 'users', 'backup']

In [23]:
def preprocess_summary(context, answer):
  text = "context: {} answer: {}".format(context, answer)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = q_tokenizer.encode_plus(text, return_tensors='pt', max_length = 382, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [44]:
def gen_question(context, answer):
  input_tokens_ids, attention_mask = preprocess_summary(context, answer)

  question_encoded = q_model.generate(input_ids=input_tokens_ids, 
                                             attention_mask=attention_mask,
                                             num_beams=5,
                                             no_repeat_ngram_size=2, 
                                             max_length=72, 
                                             early_stopping=True)
  # decode summarized token
  output = q_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [45]:
def gen_question1(answer):
  input_tokens_ids, attention_mask = preprocess_summary(summary, answer)

  question_encoded = q_model.generate(input_ids=input_tokens_ids, 
                                             attention_mask=attention_mask,
                                             num_beams=5,
                                             no_repeat_ngram_size=2, 
                                             max_length=72, 
                                             early_stopping=True)
  # decode summarized token
  output = q_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [28]:
import time

In [246]:
s = time.time()
for kw in filtered_kw:
  print(gen_question(summary, kw))
  print(kw)
e = time.time()

print(e-s)

Cloud computing services provide users with how many functions?
series
Who can use cloud computing services?
users
Along with email storage and data retrieval, what function does the cloud provide?
backup
3.97314453125


In [96]:
kw_np = np.array(filtered_kw)

In [98]:
s = time.time()
fun = np.vectorize(gen_question1)

print(fun(kw_np))
e = time.time()

['What means files are stored in a single location and it is maintained from only that location?'
 'Centralized dbms architecture requires less of what?'
 'What is stored in a single location?'
 'What will be lost if a system fails?'
 'What are the advantages of centralized dbms?'
 'How much data access traffic does centralized dbms avoid?'
 'When is there no database recovery measure?'
 'What is improved by using centralized dbms?'
 'What is a bottleneck of data access traffic?'
 'Where are files stored in centralized dbms?'
 'What is a bottleneck of centralized dbms?'
 'What is increased with centralized dbms?'
 'What does centralized dbms not provide when a system fails?'
 'If a centralized dbms architecture is slow, then it takes more time to access and search data.'
 'If the network is slow, what does it take to access and search?'
 'Centralized dbms requires less power and what else?']


In [42]:
print(e-s)

34.54746103286743
